In [12]:
from connection import get_snowpark_session

# 1Start Snowpark session
session = get_snowpark_session()

11:32:27 - INFO - Snowflake Connector for Python Version: 3.15.0, Python Version: 3.12.7, Platform: Windows-11-10.0.26100-SP0
11:32:27 - INFO - Connecting to GLOBAL Snowflake domain
11:32:27 - INFO - Snowpark Session information: 
"version" : 1.33.0,
"python.version" : 3.12.7,
"python.connector.version" : 3.15.0,
"python.connector.session.id" : 3949694875385870,
"os.name" : Windows



In [28]:
from snowflake.snowpark import Session
from snowflake.snowpark import Row
from snowflake.snowpark.types import StructType, StructField, IntegerType, StringType, TimestampType
from datetime import datetime

# Step 1: Initialize Snowpark session (replace with your actual config)
# session = Session.builder.configs(connection_parameters).create()

# Step 2: Create target table and insert initial data
session.sql("""
CREATE OR REPLACE TABLE SNOWPARK_DB.PUBLIC.CUSTOMER_DIM (
    ID INT,
    NAME STRING,
    CITY STRING,
    UPDATED_AT TIMESTAMP
)
""").collect()

session.sql("""
INSERT INTO SNOWPARK_DB.PUBLIC.CUSTOMER_DIM VALUES
    (1, 'Raj', 'Pune', '2023-12-01 10:00:00'),
    (2, 'Virat', 'Delhi', '2024-01-01 10:00:00')
""").collect()


[Row(number of rows inserted=2)]

In [29]:
#new data to be upserted


from snowflake.snowpark import Row

data = [
    Row(ID=1, NAME="Raj", CITY="Mumbai", UPDATED_AT="2024-07-01 10:00:00"),
    Row(ID=3, NAME="viraj", CITY="Delhi", UPDATED_AT="2024-07-01 10:00:00"),
    Row(ID=4, NAME="rahul", CITY="Hyderabad", UPDATED_AT="2024-07-01 10:00:00")
]

src_df = session.create_dataframe(data)
src_df.show()


---------------------------------------------------
|"ID"  |"NAME"  |"CITY"     |"UPDATED_AT"         |
---------------------------------------------------
|1     |Raj     |Mumbai     |2024-07-01 10:00:00  |
|3     |viraj   |Delhi      |2024-07-01 10:00:00  |
|4     |rahul   |Hyderabad  |2024-07-01 10:00:00  |
---------------------------------------------------



In [30]:
def upsert_scd1_snowpark(session, src_df, target_table, pk_cols):
    # Write source DataFrame to temp table
    temp_table = "TEMP_SRC"
    src_df.write.save_as_table(temp_table, mode="overwrite")

    # Generate SQL for MERGE
    on_clause = " AND ".join([f"T.{col} = S.{col}" for col in pk_cols])
    print(f"ON Clause: {on_clause}")

    # Generate update and insert clauses
    update_clause = ", ".join([f"{col} = S.{col}" for col in src_df.columns if col not in pk_cols])
    print(f"Update Clause: {update_clause}")

    # Prepare insert columns and values
    insert_cols = ", ".join(src_df.columns)
    print(f"Insert Columns: {insert_cols}")

    # Prepare insert values
    insert_values = ", ".join([f"S.{col}" for col in src_df.columns])
    print(f"Insert Values: {insert_values}")

    # Construct the MERGE SQL statement
    merge_sql = f"""
        MERGE INTO {target_table} T
        USING {temp_table} S
        ON {on_clause}
        WHEN MATCHED THEN UPDATE SET {update_clause}
        WHEN NOT MATCHED THEN INSERT ({insert_cols}) VALUES ({insert_values})
    """
    print(f"Merge SQL: {merge_sql}")

    # Execute the MERGE statement
    session.sql(merge_sql).collect()


In [31]:
target_table = "SNOWPARK_DB.PUBLIC.CUSTOMER_DIM"
pk_cols = ["ID"]

upsert_scd1_snowpark(session, src_df, target_table, pk_cols)


ON Clause: T.ID = S.ID
Update Clause: NAME = S.NAME, CITY = S.CITY, UPDATED_AT = S.UPDATED_AT
Insert Columns: ID, NAME, CITY, UPDATED_AT
Insert Values: S.ID, S.NAME, S.CITY, S.UPDATED_AT
Merge SQL: 
        MERGE INTO SNOWPARK_DB.PUBLIC.CUSTOMER_DIM T
        USING TEMP_SRC S
        ON T.ID = S.ID
        WHEN MATCHED THEN UPDATE SET NAME = S.NAME, CITY = S.CITY, UPDATED_AT = S.UPDATED_AT
        WHEN NOT MATCHED THEN INSERT (ID, NAME, CITY, UPDATED_AT) VALUES (S.ID, S.NAME, S.CITY, S.UPDATED_AT)
    


In [26]:
session.sql("""
    SELECT * 
    FROM SNOWPARK_DB.PUBLIC.CUSTOMER_DIM
    ORDER BY ID
""").show()


---------------------------------------------------
|"ID"  |"NAME"  |"CITY"     |"UPDATED_AT"         |
---------------------------------------------------
|1     |Raj     |Mumbai     |2024-07-01 10:00:00  |
|2     |Virat   |Delhi      |2024-01-01 10:00:00  |
|3     |viraj   |Delhi      |2024-07-01 10:00:00  |
|4     |rahul   |Hyderabad  |2024-07-01 10:00:00  |
---------------------------------------------------

